# LangGraph: State Machines для LLM

**Цель:** строить оркестрацию LLM через граф состояний — узлы, рёбра, циклы, ветвления.

**Что сделаем:**
- установим `langgraph`, `langchain`
- создадим минимальный StateGraph с двумя узлами
- добавим условное ребро (branching)
- покажем цикл (retry до успеха)

---

### 🎯 Якорь
LangGraph = карта маршрутов: не линейка A→B→C, а лабиринт с поворотами. Можно вернуться, разветвиться, спросить человека.

**Термины:** StateGraph | add_node | add_edge | add_conditional_edge

In [ ]:
# ← 1. Установка (Colab)
%pip install -q langgraph langchain langchain-openai langchain-community

## 1. Минимальный StateGraph: retrieve → generate

Состояние — словарь с ключом `messages`. Узлы — функции, меняющие state.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

# ← 1. Состояние графа
class State(TypedDict):
    messages: list

# ← 2. Узлы = шаги
def retrieve(state: State) -> dict:
    """Заглушка: в реальности — вызов retriever"""
    return {"messages": state["messages"] + [HumanMessage(content="[контекст из базы]")]}

def generate(state: State) -> dict:
    """Генерация ответа (заглушка без API)"""
    return {"messages": state["messages"] + [HumanMessage(content="[ответ модели]")]}

# ← 3. Собираем граф
workflow = StateGraph(State)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# ← 4. Компиляция и запуск
app = workflow.compile()
result = app.invoke({"messages": [HumanMessage(content="Что такое RAG?")]})
print("Сообщений в результате:", len(result["messages"]))
#        ↑
#        └── ✨ ВОТ ЗДЕСЬ МАГИЯ: retrieve → generate — оркестрация через граф

## 2. Условное ребро (branching)

По условию решаем: идти в узел A или B. Например: «нужен ли дополнительный retrieval?»

In [ ]:
# add_conditional_edge("generate", router_fn, {"a": "node_a", "b": "node_b"})
# router_fn(state) возвращает "a" или "b" — выбор следующего узла
# Микро-упражнение: добавь узел validate и ребро от generate к validate